In [1]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn import model_selection, preprocessing
from tqdm import tqdm
import string
import gensim.downloader as api
STOPWORDS = set(stopwords.words('english'))
import inspect
import re as re
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.utils import np_utils
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score

<ipython-input-1-fb534204403d>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
vocab_size = 45000
embedding_dim = 512
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

In [3]:
dataset = pd.read_csv('classifier_data_10.csv', header = 0, encoding= 'utf-8')
dataset.drop(['Unnamed: 2','Unnamed: 3'], axis=1, inplace = True)

In [4]:
X= dataset['description']
y = dataset['owner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1500)
X_train.shape
df = pd.DataFrame(data=[X_train, y_train], index=["description", "owner"]).T
df = df.append(pd.DataFrame(data=[X_test, y_test], index=["description", "owner"]).T)


df = df.replace({np.nan: oov_tok})
print(df.head())


                                            description  \
4576  Chrome Version       : 0.3.154.3URLs (if appli...   
3457  Currently, there are 3 suggest services that a...   
3879  We'd like to get an kernel module / autotest w...   
788   We should not have objects requiring static in...   
307   Product Version      : 0.2.149.27URLs (if appl...   

                        owner  
4576         jon@chromium.org  
3457       js...@chromium.org  
3879  davidri...@chromium.org  
788     jhawkins@chromium.org  
307       eroman@chromium.org  


In [5]:

df['char_count'] = df['description'].apply(len) # Number of characters in the string
df['word_count'] = df['description'].apply(lambda x: len(x.split())) # Number of words in the string 
df['word_density'] = df['char_count'] / (df['word_count']+1) # Density of word (in char)
df['title_word_count'] = df['description'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))

labels = df['owner'].unique()

def remove(list):
    pattern='[0-9]'
    list = [re.sub(pattern, '',i) for i in list]
    return list

In [6]:
train_x, valid_x, y_train, y_valid = train_test_split(df['description'], df['owner'], random_state=42, test_size=0.2)

token = Tokenizer()
token.fit_on_texts(df['description'])
word_index = token.word_index

df['description']= remove(df['description'])
encoder = preprocessing.LabelEncoder()
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=300)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=300)


train_y = encoder.fit_transform(y_train)
valid_y = encoder.fit_transform(y_valid)

 
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', patience=3)
def create_conv_model(word_index, label, embedding_matrix, pre_trained=False):
    '''
    Function to generate a convulational neural network for binary or multiclass classification.
    @param word_index: (matrix) unique token in corpus
    @param label: (list) list of labels to determine if it,s a binary or multiclass
    @param embedding_matrix: (matrix) matrix of integer for each word in the 
    @param pre_trained: (bool) determine if the model will use pretrained model
    @return: (model) convulational neural network 
    '''
    if pre_trained==False:
        embedded = keras.layers.Embedding(len(word_index) +1, 100)
    else:
        embedded = keras.layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)
    
    model = keras.Sequential([
    embedded,
    keras.layers.Conv1D(100, 5, activation='relu'), # padding='same'
    keras.layers.Dropout(0.2),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Conv1D(64, 5, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Conv1D(32, 5, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.GlobalMaxPooling1D(),

    keras.layers.Dense(1 if len(label)<=2 else len(label), activation='sigmoid' if len(label)<=2 else "softmax")])

    if len(label)==2:
        model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    else: 
        model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model

In [7]:
model = create_conv_model(word_index, labels, False)

In [10]:
history = model.fit(train_seq_x, train_y, epochs=20, callbacks=[es], validation_split=0.2)

Epoch 1/20
100/100 [==============================] - 8s 81ms/step - loss: 6.4452 - accuracy: 0.0100 - val_loss: 6.4758 - val_accuracy: 0.0063
Epoch 2/20
100/100 [==============================] - 8s 80ms/step - loss: 6.2511 - accuracy: 0.0281 - val_loss: 6.3925 - val_accuracy: 0.0288
Epoch 3/20
100/100 [==============================] - 8s 79ms/step - loss: 6.1403 - accuracy: 0.0372 - val_loss: 6.3458 - val_accuracy: 0.0288
Epoch 4/20
100/100 [==============================] - 8s 80ms/step - loss: 6.0640 - accuracy: 0.0372 - val_loss: 6.3040 - val_accuracy: 0.0288
Epoch 5/20
100/100 [==============================] - 8s 80ms/step - loss: 6.0113 - accuracy: 0.0372 - val_loss: 6.2787 - val_accuracy: 0.0288
Epoch 6/20
100/100 [==============================] - 8s 79ms/step - loss: 5.9675 - accuracy: 0.0372 - val_loss: 6.2450 - val_accuracy: 0.0288
Epoch 7/20
100/100 [==============================] - 8s 79ms/step - loss: 5.9332 - accuracy: 0.0372 - val_loss: 6.2386 - val_accuracy: 0.0288

In [9]:
results = model.evaluate(valid_seq_x, valid_y)

print(results)


32/32 [==============================] - 1s 15ms/step - loss: 6.6290 - accuracy: 0.0010
[6.6289801597595215, 0.0010000000474974513]
